In [2]:
import arcgis
from arcgis import GIS
from arcgis.geometry import Point
from arcgis.geometry import Geometry
import pandas as pd
import json

In [3]:
# Load data
data = pd.read_csv("data.csv")

In [4]:
# Object gis for autentication
gis = GIS("https://www.arcgis.com", "danndch", "046Aebf2!7709")

In [ ]:
# Create a list to hold features
features = []

In [ ]:
# Iterate over each row in the DataFrame
for _, row in data.iterrows():
    # Create a dictionary to hold feature attributes
    attributes = {
        "neighborhood_name": row["neighborhood_name"],
        "population": row["population"],
        "year": row["year"]
    }
    
    # Create a Point geometry
    geometry = Geometry(
        {
            "x": row["longitude"],
            "y": row["latitude"],
            "spatialReference": {"wkid": 4326}
        }
    )
    
    # Create a feature dictionary
    feature = {
        "attributes": attributes,
        "geometry": geometry
    }
    
    # Append the feature to the list
    features.append(feature)


In [ ]:
# Create a feature collection
feature_collection = {
    "featureCollection": {
        "layers": [
            {
                "layerDefinition": {
                    "geometryType": "esriGeometryPoint",
                    "objectIdField": "OBJECTID",
                    "fields": [
                        {
                            "name": "neighborhood_name",
                            "type": "esriFieldTypeString",
                            "alias": "Neighborhood Name"
                        },
                        {
                            "name": "population",
                            "type": "esriFieldTypeInteger",
                            "alias": "Population"
                        },
                        {
                            "name": "year",
                            "type": "esriFieldTypeInteger",
                            "alias": "Year"
                        }
                    ]
                },
                "featureSet": {
                    "features": features,
                    "geometryType": "esriGeometryPoint"
                }
            }
        ]
    }
}


In [ ]:
# Convert feature_collection to JSON string
feature_collection_json = json.dumps(feature_collection)


In [ ]:
# Publish the feature collection as a hosted feature layer
item_properties = {
    "title": "Neighborhoods",
    "text": "Neighborhoods Feature Layer",
    "type": "Feature Collection",
    "tags": "neighborhoods, data"
}
item = gis.content.add(item_properties, data=feature_collection_json)
feature_layer = item.publish()

In [ ]:
# Visualize the feature layer on a map
map = gis.map("Alaska")
map.add_layer(feature_layer)
map